In [ ]:
pip install pandas

In [ ]:
pip install tensorflow

In [ ]:
pip install pytesseract 

In [ ]:
pip install plotly.express

In [1]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install scikit-learn

In [ ]:
pip install

In [ ]:
pip install

In [ ]:
pip install

In [ ]:
pip install

In [ ]:
pip install

In [ ]:
pip install

In [ ]:
pip install

In [ ]:
pip install

In [ ]:
pip install

In [43]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [19]:
# labeling data sudah jadi csv nya itu dataset kita

In [ ]:
#

# 1. Data Processing

In [69]:
# df = pd.read_csv("/Users/rozidatululummiah/Downloads/labels.csv")

In [70]:
# df


,filepath,xmin,xmax,ymin,ymax
0,/Users/asik/Desktop/ANPR/images/N517.xml,175,290,228,255
1,/Users/asik/Desktop/ANPR/images/N271.xml,262,318,141,160
2,/Users/asik/Desktop/ANPR/images/N265.xml,87,216,166,212
3,/Users/asik/Desktop/ANPR/images/N503.xml,66,174,302,332
4,/Users/asik/Desktop/ANPR/images/N259.xml,324,475,166,205
...,...,...,...,...,...
471,/Users/asik/Desktop/ANPR/images/N254.xml,166,235,175,207
472,/Users/asik/Desktop/ANPR/images/N532.xml,179,347,256,290
473,/Users/asik/Desktop/ANPR/images/N268.xml,213,271,133,168
474,/Users/asik/Desktop/ANPR/images/N297.xml,158,247,149,170


In [10]:
path = glob('/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [11]:
df = pd.DataFrame(labels_dict)

In [12]:
df

,filepath,xmin,xmax,ymin,ymax
0,/Users/rozidatululummiah/Downloads/Automatic-L...,207,356,174,287
1,/Users/rozidatululummiah/Downloads/Automatic-L...,39,108,129,157
2,/Users/rozidatululummiah/Downloads/Automatic-L...,289,421,188,232
3,/Users/rozidatululummiah/Downloads/Automatic-L...,150,378,217,269
4,/Users/rozidatululummiah/Downloads/Automatic-L...,208,370,89,173
...,...,...,...,...,...
220,/Users/rozidatululummiah/Downloads/Automatic-L...,217,428,147,188
221,/Users/rozidatululummiah/Downloads/Automatic-L...,787,910,473,513
222,/Users/rozidatululummiah/Downloads/Automatic-L...,342,494,243,288
223,/Users/rozidatululummiah/Downloads/Automatic-L...,164,285,99,137


In [13]:

# Simpan DataFrame ke file CSV di direktori yang ditentukan
output_directory = '/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main'
os.makedirs(output_directory, exist_ok=True)
df.to_csv(os.path.join(output_directory, 'labels.csv'), index=False)


In [14]:
df

,filepath,xmin,xmax,ymin,ymax
0,/Users/rozidatululummiah/Downloads/Automatic-L...,207,356,174,287
1,/Users/rozidatululummiah/Downloads/Automatic-L...,39,108,129,157
2,/Users/rozidatululummiah/Downloads/Automatic-L...,289,421,188,232
3,/Users/rozidatululummiah/Downloads/Automatic-L...,150,378,217,269
4,/Users/rozidatululummiah/Downloads/Automatic-L...,208,370,89,173
...,...,...,...,...,...
220,/Users/rozidatululummiah/Downloads/Automatic-L...,217,428,147,188
221,/Users/rozidatululummiah/Downloads/Automatic-L...,787,910,473,513
222,/Users/rozidatululummiah/Downloads/Automatic-L...,342,494,243,288
223,/Users/rozidatululummiah/Downloads/Automatic-L...,164,285,99,137


In [19]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images',filename_image)
    return filepath_image
getFilename(filename)

'/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/N107.jpeg'

In [20]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]#random check

['/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/N107.jpeg',
 '/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/N113.jpeg',
 '/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/N62.jpeg',
 '/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/N89.jpeg',
 '/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/N88.jpeg',
 '/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/N63.jpeg',
 '/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/N77.jpeg',
 '/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/N112.jpeg',
 '/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/N106.jpeg',
 '/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/images/N110.jpeg']

In [21]:
#Targeting all our values in array selecting all columns
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [22]:
# Convert data to array
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [23]:
# Split the data into training and testing set using sklearn.
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape


((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

# 2. DEEP LEARNING FOR OBJECT DETECTION 

In [24]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)


# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

219055592/219055592 [==============================] - 67s 0us/step


In [25]:
# Complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

In [35]:
tfb = TensorBoard(log_dir='/Users/rozidatululummiah/Downloads/Tensorflow/models/research/object_detection')
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=180,
                    validation_data=(x_test,y_test),callbacks=[tfb])



Epoch 1/180
18/18 [==============================] - 64s 3s/step - loss: 0.0370 - val_loss: 0.0291
Epoch 2/180
18/18 [==============================] - 49s 3s/step - loss: 0.0168 - val_loss: 0.0192
Epoch 3/180
18/18 [==============================] - 52s 3s/step - loss: 0.0091 - val_loss: 0.0124
Epoch 4/180
18/18 [==============================] - 55s 3s/step - loss: 0.0045 - val_loss: 0.0126
Epoch 5/180
18/18 [==============================] - 57s 3s/step - loss: 0.0029 - val_loss: 0.0112
Epoch 6/180
18/18 [==============================] - 58s 3s/step - loss: 0.0025 - val_loss: 0.0106
Epoch 7/180
18/18 [==============================] - 61s 3s/step - loss: 0.0019 - val_loss: 0.0113
Epoch 8/180
18/18 [==============================] - 59s 3s/step - loss: 0.0020 - val_loss: 0.0100
Epoch 9/180
18/18 [==============================] - 56s 3s/step - loss: 0.0015 - val_loss: 0.0088
Epoch 10/180
18/18 [==============================] - 56s 3s/step - loss: 0.0013 - val_loss: 0.0089
Epoch 11/

In [45]:
model.save('/Users/rozidatululummiah/Downloads/Tensorflow/models/research/object_detection.h5')
print('Model loaded Sucessfully')

Model loaded Sucessfully


In [46]:
model = tf.keras.models.load_model('/Users/rozidatululummiah/Downloads/Tensorflow/models/research/object_detection.h5')
print('Model loaded Sucessfully')

Model loaded Sucessfully


In [47]:
path = '/Users/rozidatululummiah/Downloads/Automatic-License-Plate-Detection-main/TEST/TEST_indo.jpeg'
image = load_img(path) # PIL object
image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
image1 = load_img(path,target_size=(224,224))
image_arr_224 = img_to_array(image1)/255.0  # Convert into array and get the normalized output

# Size of the orginal image
h,w,d = image.shape
print('Height of the image =',h)
print('Width of the image =',w)

Height of the image = 148
Width of the image = 340


In [48]:
fig = px.imshow(image)
fig.update_layout(width=700, height=500,  margin=dict(l=10, r=10, b=10, t=10), xaxis_title='Figure 13 - TEST Image')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'x: %{x}<br>y: %{y}<br>color: [%{z[0]}, %{z[1]}, %{z[2]}]<extra></extra>',
              'name': '0',
              'source': ('data:image/png;base64,iVBORw0K' ... '8BL88abZZuiUMAAAAASUVORK5CYII='),
              'type': 'image',
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'height': 500,
               'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10},
               'template': '...',
               'width': 700,
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Figure 13 - TEST Image'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0]}}
})